In [1]:
import requests    
import re
import json
from lxml import html
import numpy as np
import pandas as pd

In [2]:
base_url = 'https://dblp.org/pid/21/7664.html'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

In [3]:
requests.adapters.DEFAULT_RETRIES = 5

In [4]:
s = requests.session()
s.keep_alive = False

In [25]:
response = requests.get(base_url, verify=False)
data = response.text

D:\Tools\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dblp.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [26]:
tree=html.fromstring(data) 

以下为数据获取操作

In [27]:
paper_type= np.array(tree.xpath('//div[@class="nr"]/text()')) #获取需要的数据
paper_box = np.array(tree.xpath('//div[@class="box"]'))

In [29]:
len(paper_type) # 获取的文章总数

326

In [30]:
# 获取满足条件的项idx
index = []
for idx,pt in enumerate(paper_type):
    if 'j' in pt or 'c' in pt:
        index.append(idx)    

In [86]:
len(index)

211

In [94]:
paper_title = np.array(tree.xpath('//cite[@class="data tts-content"]/span[@class="title"and@itemprop="name"]/text()'))
paper_authors = np.array(tree.xpath('//li//cite[@class="data tts-content"]//span[@itemprop="author"][position()<2]//span/text()'))
paper_year = np.array(tree.xpath('//cite[@class="data tts-content"]//span[@itemprop="datePublished"]/text()'))
paper_conference = np.array(tree.xpath('//cite[@class="data tts-content"]//a//span[@itemprop="isPartOf"]/span[@itemprop="name"]/text()'))
paper_url = np.array(tree.xpath('//nav[@class="publ"]//ul//li[@class="drop-down"][1]//div[@class="head"]/a/@href'))

In [95]:
print([len(paper_title), len(paper_authors), len(paper_year), len(paper_conference), len(paper_url)])

[327, 326, 326, 325, 326]


（1）paper_conference少一年因为b1没有期刊

In [96]:
paper_conference = np.insert(paper_conference,-16, 0)

In [97]:
print([len(paper_title), len(paper_authors), len(paper_year), len(paper_conference), len(paper_url)])

[327, 326, 326, 326, 326]


(2)ERL-Re:..... 奇怪的title分成了两个

In [98]:
for idx,name in enumerate(paper_title):
    if name == "ERL-Re":
        paper_title[idx] += paper_title[idx+1]
        paper_title = np.delete(paper_title, idx+1)
        break

In [99]:
print([len(paper_title), len(paper_authors), len(paper_year), len(paper_conference), len(paper_url)])

[326, 326, 326, 326, 326]


In [101]:
# 获取数据
paper_title = paper_title[index]
paper_year = paper_year[index]
paper_conference = paper_conference[index]
paper_url = paper_url[index]
paper_authors = paper_authors[index]

In [102]:
data_frame = np.array([paper_year, paper_title, paper_conference, paper_authors, paper_url]).T

In [103]:
print([len(paper_title), len(paper_authors), len(paper_year), len(paper_conference), len(paper_authors), len(paper_url)])

[211, 211, 211, 211, 211, 211]


全部作者再另外一个板块实现，此代码的全部作者为文章的第一作者

In [104]:
column_names = ["年份", "论文名", "会议简称", "全部作者", "发表链接"]
data_df = pd.DataFrame(data_frame, columns=column_names)

In [105]:
data_df

,年份,论文名,会议简称,全部作者,发表链接
0,2023,Accelerating deep reinforcement learning via k...,Auton. Agents Multi Agent Syst.,Yuanqiang Yu,https://doi.org/10.1007/s10458-023-09600-1
1,2023,A benchmark for automatic medical consultation...,Bioinform.,Wei Chen,https://doi.org/10.1093/bioinformatics/btac817
2,2023,A Unified Framework for Layout Pattern Analysi...,IEEE Trans. Comput. Aided Des. Integr. Circuit...,Ran Chen,https://doi.org/10.1109/TCAD.2022.3192363
3,2023,Contrastive-ACE: Domain Generalization Through...,IEEE Trans. Image Process.,Yunqi Wang,https://doi.org/10.1109/TIP.2022.3227457
4,2023,Structure Aware Incremental Learning with Pers...,AAAI,Yuening Wang,https://ojs.aaai.org/index.php/AAAI/article/vi...
...,...,...,...,...,...
206,2012,Maintaining cooperation in homogeneous multi-a...,SMC,Jianye Hao,https://doi.org/10.1109/ICSMC.2012.6377718
207,2011,Learning to Achieve Social Rationality Using T...,ICTAI,Jianye Hao,https://doi.org/10.1109/ICTAI.2011.31
208,2010,Strategy and Fairness in Repeated Two-agent In...,ICTAI (2),Jianye Hao,https://doi.org/10.1109/ICTAI.2010.75
209,2009,Bus-Based and NoC Infrastructure Performance E...,ITNG,Ling Wang,https://doi.org/10.1109/ITNG.2009.186


In [106]:
data_df.to_excel(excel_writer='tab1.xlsx', sheet_name='sheet_1')